# Importandos as bibliotecas

In [1]:
import os
import requests
import zipfile
import pandas as pd

# Baixando o arquivo no diretório existente

In [20]:
url = 'https://leandrolessa.com.br/wp-content/uploads/2024/03/pesquisa_preferencias.zip'
resp = requests.get(url)

dir_dest = 'D:\\XPE-Post_Grad\\Projects\\datasets'
if resp.status_code == requests.codes.OK:
    nome_arqzip = url.split('/')[-1]
    cam_arqzip = os.path.join(dir_dest, nome_arqzip)
    with open(cam_arqzip, 'wb') as arquivo:
        arquivo.write(resp.content)
        print('Download efetuado com sucesso! Salvo em {}.'.format(cam_arqzip))

In [19]:
with zipfile.ZipFile(cam_arqzip, 'r') as zip_ref:
    zip_ref.extractall(dir_dest)
print(f"Arquivo ZIP descompactado em: {dir_dest}")

Arquivo ZIP descompactado em: D:\XPE-Post_Grad\Projects\datasets


# Carregando os dados dos arquivos para um dataframe

In [2]:
PATH = 'D:/XPE-Post_Grad/Projects/datasets/'
lista_path = os.listdir(PATH)

In [3]:
lista_df = []
for arquivo in lista_path:
    local_arquivo = os.path.join(PATH, arquivo)
    if local_arquivo.endswith('.csv'):
        df = pd.read_csv(local_arquivo, sep='|', encoding='latin-1')
        lista_df.append(df)

In [4]:
df = pd.concat(lista_df)

In [5]:
df.head()

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte
1,10141860,2020-01-01,Feminino,1966-01-19,peixe,quente,Café,Assistir TV
2,10142548,2020-01-01,Masculino,1973-08-02,tartaruga,frio,Café,Ler livros
3,10142591,2020-01-01,Masculino,1997-03-05,gato,moderado,Cerveja,Aprender algo novo
4,10142654,2020-01-01,Feminino,1969-08-05,tartaruga,moderado,Café,Assistir TV


# Separando os dataframes para corresponderem com as tabelas

In [6]:
animais = list(df.animal_estimacao.unique())
animais

['peixe', 'tartaruga', 'gato', 'cachorro']

In [7]:
bebidas = list(df.bebida_favorita.unique())
bebidas

['Cerveja', 'Café', 'refrigerante', 'Água', 'Vinho', 'Chá']

In [8]:
hobbies = list(df.hobbies.unique())
hobbies

['Praticar esporte',
 'Assistir TV',
 'Ler livros',
 'Aprender algo novo',
 'Escrever',
 'Pintar quadros',
 'Dormir',
 'Escutar música']

In [9]:
climas = list(df.clima.unique())
climas

['frio', 'quente', 'moderado']

In [10]:
data_ref = pd.to_datetime('2024-03-05')
df['data_nascimento'] = df['data_nascimento'].astype('datetime64[ns]')
df['idade'] = (data_ref - df['data_nascimento']).astype('<m8[Y]')
df['idade'] = df['idade'].astype('int64')

In [11]:
df.head()

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies,idade
0,10141743,2020-01-01,Masculino,1981-04-25,peixe,frio,Cerveja,Praticar esporte,42
1,10141860,2020-01-01,Feminino,1966-01-19,peixe,quente,Café,Assistir TV,58
2,10142548,2020-01-01,Masculino,1973-08-02,tartaruga,frio,Café,Ler livros,50
3,10142591,2020-01-01,Masculino,1997-03-05,gato,moderado,Cerveja,Aprender algo novo,27
4,10142654,2020-01-01,Feminino,1969-08-05,tartaruga,moderado,Café,Assistir TV,54


In [12]:
df_pessoas = df[['cod_pessoa', 'genero', 'data_nascimento', 'idade']]
df_pessoas

,cod_pessoa,genero,data_nascimento,idade
0,10141743,Masculino,1981-04-25,42
1,10141860,Feminino,1966-01-19,58
2,10142548,Masculino,1973-08-02,50
3,10142591,Masculino,1997-03-05,27
4,10142654,Feminino,1969-08-05,54
...,...,...,...,...
1,10142475,Masculino,1955-05-30,68
2,10142522,Masculino,1954-12-22,69
3,10142881,Masculino,1952-07-30,71
4,10143184,Masculino,1984-06-05,39


In [15]:
df_pessoas.duplicates().sum()

AttributeError: 'DataFrame' object has no attribute 'duplicates'

# Inserindo os dados dos dataframes no Banco de Dados coleta_pericles

In [9]:
!pip install mysqlclient

In [10]:
import MySQLdb as mysqldb
import sqlalchemy as sqlal

In [14]:
usuario = 'pericles@localhost'
senha = 'Access2Da1aBas3'
servidor = 'localhost'
base_de_dados = 'coleta_pericles'

conexao = f'mysql://{usuario}:{senha}@{servidor}/{base_de_dados}'
conexao

'mysql://pericles@localhost:Access2Da1aBas3@localhost/coleta_pericles'

In [15]:
engine = sqlal.create_engine(conexao)
conn = engine.connect()

In [ ]:
for pessoa in df_pessoas.itertuples():
    cod_pessoa = pessoa.cod_pessoa
    genero = pessoa.genero
    data_nascimento = str(pessoa.data_nascimento)
    idade = pessoa.idade
    try:
        query = f"""INSERT INTO pessoa (cod_pessoa, genero, data_nascimento, idade) values ('{cod_pessoa}', '{genero}', '{data_nascimento}', '{idade}')"""
        conn.execute(query)
        print(f'Registro executado com sucesso para {cod_pessoa}')
    except Exception as e:
        print(f'Erro {e} para o registro de {cod_pessoa}')

Registro executado com sucesso para 10141743
Registro executado com sucesso para 10141860
Registro executado com sucesso para 10142548
Registro executado com sucesso para 10142591
Registro executado com sucesso para 10142654
Registro executado com sucesso para 10141523
Registro executado com sucesso para 10141996
Registro executado com sucesso para 10142089
Registro executado com sucesso para 10142145
Registro executado com sucesso para 10142520
Registro executado com sucesso para 10143160
Registro executado com sucesso para 10141712
Registro executado com sucesso para 10141987
Registro executado com sucesso para 10142093
Registro executado com sucesso para 10142325
Registro executado com sucesso para 10142552
Registro executado com sucesso para 10142598
Registro executado com sucesso para 10143013
Registro executado com sucesso para 10141570
Registro executado com sucesso para 10141851
Registro executado com sucesso para 10141938
Registro executado com sucesso para 10142733
Registro e

In [45]:
def insercao(nome_lista, nome_tabela, entrada):
    for item in nome_lista:
        try:
            query = f"""INSERT INTO {nome_tabela} ({entrada}) values ('{item}')"""
            conn.execute(query)
            print(f'Sucesso para {item}')
        except Exception as e:
            print(f'Erro {e} para {item}')

In [46]:
insercao(animais, 'animal_estimacao', 'nome_animal')

Sucesso para peixe
Sucesso para tartaruga
Sucesso para gato
Sucesso para cachorro


In [47]:
insercao(hobbies, 'hobby', 'nome_hobby')

Sucesso para Praticar esporte
Sucesso para Assistir TV
Sucesso para Ler livros
Sucesso para Aprender algo novo
Sucesso para Escrever
Sucesso para Pintar quadros
Sucesso para Dormir
Sucesso para Escutar música


In [48]:
insercao(bebidas, 'bebida', 'nome_bebida')

Sucesso para Cerveja
Sucesso para Café
Sucesso para refrigerante
Sucesso para Água
Sucesso para Vinho
Sucesso para Chá


In [49]:
insercao(climas, 'clima', 'descr_clima')

Sucesso para frio
Sucesso para quente
Sucesso para moderado
